In [91]:
# Первій Михайло КА-96 20 Вар
# Реалізувати метод Якобі

import numpy as np
import math
EPS = 10**-5


A = [[6.26, 0.94, 1.13, 1.24],
     [0.94, 4.16, 1.3,  0.16],
     [1.13,  1.3,  5.44,  2.1],
     [1.24, 0.16, 2.1,   6.1]]

A = np.array(A)

y_0 = [1,1,1,1]

In [92]:
# Часткова задача (Степеневим методом)
def vnorm(vec: list):
    # vector norm
    sum = 0
    for i in range(len(vec)):
        sum += vec[i]**2
    return sum ** 0.5

def matrix_pow(M, n):
    res = np.identity(len(M))
    for i in range(n):
        res = np.matmul(M, res)
    return res

def matrix_print(M):
    for i in range(len(M)):
        print("|", end="  ")
        for j in range(len(M[i])):
            print("%11.7f" % (M[i][j]) , end="  ")
        print("|")


def solvePart(A, max=True):
    if not max:
        A = np.linalg.inv(A)
        
    print("Starting partial method on matrix A:")
    matrix_print(A)
    print()
    
    print(" "*25, "eigenvalue radius")
    y_0 = [1]*len(A)
    
    y_cur = np.matmul(A, y_0)
    y_next = np.matmul(A, y_cur)
    
    
    lam = np.divide(y_cur,y_0)
    print("Lambda 0", lam)
    
    lam_next = np.divide(y_next,y_cur)
    print("lambda 1", lam_next)
    
    i = 2
    while vnorm(lam_next - lam) >= EPS:
        y_cur = y_next
        y_next = np.matmul(A, y_cur)
        
        lam = lam_next
        lam_next = np.divide(y_next, y_cur)
        print("Lambda", i, lam_next)
        i += 1
    
    result = 0
    
    for j in lam_next:
        result += j
    result /= len(lam_next)
    if not max:
        result = 1/result
        eigenvector = y_next * (result ** i)
    else:
        eigenvector = y_next/(result ** i)
    
    return result, eigenvector

res_max, eigenMax = solvePart(A)
print("Calculated spectral radius (max eigenvalue): ", res_max)
print("Vector assosiated: ", eigenMax/max(eigenMax))
print("------------------------------")
res_min, eigenMin = solvePart(A, False)
print("Calculated inverse spectral radius (min eigenvalue): ", res_min)
print("Vector assosiated: ", eigenMin/max(eigenMin))

Starting partial method on matrix A:
|    6.2600000    0.9400000    1.1300000    1.2400000  |
|    0.9400000    4.1600000    1.3000000    0.1600000  |
|    1.1300000    1.3000000    5.4400000    2.1000000  |
|    1.2400000    0.1600000    2.1000000    6.1000000  |

                          eigenvalue radius
Lambda 0 [9.57 6.56 9.97 9.6 ]
lambda 1 [9.32546499 7.74121951 9.40209629 9.62639583]
Lambda 2 [9.26581468 8.50277884 9.29039478 9.51552526]
Lambda 3 [9.25951999 8.90798604 9.27801143 9.42434564]
Lambda 4 [9.2667347  9.10682669 9.28361106 9.36735511]
Lambda 5 [9.27533084 9.20255833 9.28930953 9.33467949]
Lambda 6 [9.28217109 9.2489497  9.29264659 9.31658962]
Lambda 7 [9.28693625 9.27175005 9.29424259 9.30672314]
Lambda 8 [9.29004822 9.2831207  9.29489619 9.30137125]
Lambda 9 [9.29200609 9.28886548 9.29511148 9.29846998]
Lambda 10 [9.29320873 9.29180054 9.2951478  9.29689409]
Lambda 11 [9.29393539 9.29331473 9.2951235  9.29603526]
Lambda 12 [9.29436925 9.2941027  9.29508689 9.295565

In [105]:
# Метод поворотів Якобі (20 вар)

def final_condition(M):
    """
    Returns true if sqrt of sum of squared is less then goal estimation EPS
    """
    s = 0 # sum
    for i in range(len(M)):
        for j in range(len(M[i])):
            if i < j:
                s += M[i][j] ** 2
    return math.sqrt(s) < EPS
    
    

def Jacobi(M):
    """
    Uses Jacobi rotation method to calculate eigenvalues & eigenvectors
    :return: list of eigenvalues
     """
    
    EVec = np.identity(len(M))
    
    print("Jacobi stated with matrix A")
    matrix_print(M)
    print()
    
    A_cur = M
    i_max = j_max = 0
    el_max = np.NINF
    
    for i in range(len(A_cur)):
        for j in range(len(A_cur[i])):
            if i < j and abs(A_cur[i][j]) > el_max:
                el_max = abs(A_cur[i][j])
                i_max = i
                j_max = j
    print("Max element absolute =", el_max, " on position ", (i_max+1, j_max+1))
    
    U = np.identity(len(M))
    U_T = np.identity(len(M))
    if A[i_max][i_max] != A[j_max][j_max]:
        phi = 0.5 * math.atan(2 * A_cur[i_max][j_max] / (A_cur[i_max][i_max] - A_cur[j_max][j_max]))
    else:
        phi = math.pi/4

    U[i_max][i_max] = U[j_max][j_max] = U_T[i_max][i_max] = U_T[j_max][j_max] = math.cos(phi)
    U[i_max][j_max] = U_T[j_max][i_max] = -math.sin(phi)
    U[j_max][i_max] = U_T[i_max][j_max] = math.sin(phi)
    EVec = np.matmul(EVec, U)
    
    A_next = np.matmul(np.matmul(U_T, A_cur), U)
    print("\nIteration # 1")
    matrix_print(A_next)
    
    iteration = 2
    while not final_condition(A_next):
        A_cur = A_next
        i_max = j_max = 0
        el_max = np.NINF

        for i in range(len(A_cur)):
            for j in range(len(A_cur[i])):
                if i < j and abs(A_cur[i][j]) > el_max:
                    el_max = abs(A_cur[i][j])
                    i_max = i
                    j_max = j
        print("Max element absolute =", el_max, "on position", (i_max+1, j_max+1))
        print("a_ij=", A_cur[i_max][j_max], " a_ii=", A_cur[i_max][i_max], " a_jj=", A_cur[j_max][j_max])

        U = np.identity(len(M))
        U_T = np.identity(len(M))

        phi = 0.5 * math.atan(2 * A_cur[i_max][j_max] / (A_cur[i_max][i_max] - A_cur[j_max][j_max]))

        U[i_max][i_max] = U[j_max][j_max] = U_T[i_max][i_max] = U_T[j_max][j_max] = math.cos(phi)
        U[i_max][j_max] = U_T[j_max][i_max] = -math.sin(phi)
        U[j_max][i_max] = U_T[i_max][j_max] = math.sin(phi)
        EVec = np.matmul(EVec, U)

        A_next = np.matmul(np.matmul(U_T, A_cur), U)
        print("\nIteration #", iteration)
        matrix_print(A_next)
        iteration += 1
    res = []
    for i in range(len(A_next)):
        res.append(A_next[i][i])
    return res, EVec

eigenvalues, eigenvectors = Jacobi(A)
print("All eigenvalues of A =", eigenvalues)
print("Matrix of eigenvectors as colums")
matrix_print(eigenvectors)
print("Checking...")
v1, v2, v3, v4 = np.transpose(eigenvectors)
print("A*v1 - lambda*v1 = ", (np.matmul(A, v1) - eigenvalues[0]*v1) )
print("A*v2 - lambda*v2 = ", (np.matmul(A, v2) - eigenvalues[1]*v2) )
print("A*v3 - lambda*v3 = ", (np.matmul(A, v3) - eigenvalues[2]*v3) )
print("A*v4 - lambda*v4 = ", (np.matmul(A, v4) - eigenvalues[3]*v4) )

Jacobi stated with matrix A
|    6.2600000    0.9400000    1.1300000    1.2400000  |
|    0.9400000    4.1600000    1.3000000    0.1600000  |
|    1.1300000    1.3000000    5.4400000    2.1000000  |
|    1.2400000    0.1600000    2.1000000    6.1000000  |

Max element absolute = 2.1  on position  (3, 4)

Iteration # 1
|    6.2600000    0.9400000    0.0529275    1.6768121  |
|    0.9400000    4.1600000    0.8840308    0.9664831  |
|    0.0529275    0.8840308    3.6442296    0.0000000  |
|    1.6768121    0.9664831    0.0000000    7.8957704  |
Max element absolute = 1.676812058573542 on position (1, 4)
a_ij= 1.676812058573542  a_ii= 6.26  a_jj= 7.895770448566825

Iteration # 2
|    5.2122392    0.2850229    0.0448854    0.0000000  |
|    0.2850229    4.1600000    0.8840308    1.3177448  |
|    0.0448854    0.8840308    3.6442296    0.0280468  |
|    0.0000000    1.3177448    0.0280468    8.9435312  |
Max element absolute = 1.3177448450652627 on position (2, 4)
a_ij= 1.3177448450652627  a